# Radon-Transform (CUDA)

## Setup

In [1]:
from math import sin, cos, ceil
import typing
import torch
import torchvision
import matplotlib.pyplot as plt
import radon
import torch_radon

In [2]:
positions = torch.arange(-ceil(32*1.41421356237/2.0), ceil(32*1.41421356237/2.0)+1, dtype=torch.float32, device="cuda")
angles = torch.linspace(0.0, torch.pi, 33, device="cuda")[:-1]
a = torch.randn((1,1,32,32), device="cuda")
b = torch.randn((1,1,angles.shape[0],positions.shape[0]), device="cuda")
Aa = radon.radon_forward(a, angles, positions)
Atb = radon.radon_backward(b, 32, angles, positions)
assert torch.abs(torch.sum(Aa*b)-torch.sum(a*Atb)) <= 1e-1, "Backward transform is not transpose of forward"

AssertionError: Backward transform is not transpose of forward

In [ ]:
positions = torch.arange(-ceil(32*1.41421356237/2.0), ceil(32*1.41421356237/2.0)+1, dtype=torch.float32)
angles = torch.linspace(0.0, torch.pi, 33)[:-1]
torchradon = torch_radon.Radon(32, angles, positions.shape[0], 1)
a = torch.randn((1,1,32,32), device="cuda")
b = torch.randn((1,1,angles.shape[0],positions.shape[0]), device="cuda")
Aa = torchradon.forward(a)
Atb = torchradon.backprojection(b)
print(torch.sum(Aa*b))
print(torch.sum(a*Atb))

In [ ]:
dataset = torchvision.datasets.Caltech256("/data/datasets/", download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Grayscale(), torchvision.transforms.Resize(512, antialias=True), torchvision.transforms.CenterCrop(512)]))
rows, cols = 10, 7
for i in range(10):
    img = dataset[i][0].unsqueeze(0).contiguous().to("cuda")
    sino = radon.radon_forward(img)
    fsino = radon.radon_filter(sino, radon.ram_lak_filter)
    recon = radon.radon_backward(fsino, img.shape[-1])
    assert torch.abs(recon.min()-img.min()) <= 1e-1, f"Minimum not content invariant: {img.min()} vs {recon.min()}"
    assert torch.abs(recon.max()-img.max()) <= 1e-1, f"Maximum not content invariant: {img.max()} vs {recon.max()}"

In [ ]:
rows, cols = 10, 7
for i in range(5):
    size = [64,128,256,512,1024][i]
    img = next(iter(torchvision.datasets.Caltech256("/data/datasets/", download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Grayscale(), torchvision.transforms.Resize(size, antialias=True), torchvision.transforms.CenterCrop(size)]))))[0].unsqueeze(0).contiguous().to("cuda")
    sino = radon.radon_forward(img)
    fsino = radon.radon_filter(sino, radon.ram_lak_filter)
    recon = radon.radon_backward(fsino, img.shape[-1])
    assert torch.abs(recon.min()-img.min()) <= 1e-1, f"Minimum not resolution invariant: {img.min()} vs {recon.min()}"
    assert torch.abs(recon.max()-img.max()) <= 1e-1, f"Maximum not resolution invariant: {img.max()} vs {recon.max()}"

In [ ]:
img = next(iter(torchvision.datasets.Caltech256("/data/datasets/", download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Grayscale(), torchvision.transforms.Resize(size, antialias=True), torchvision.transforms.CenterCrop(size)]))))[0].unsqueeze(0).contiguous().to("cuda")
rows, cols = 10, 7
for i in range(7):
    positions = [
        torch.arange(-ceil(img.shape[2]*1.41421356237/2.0), ceil(img.shape[2]*1.41421356237/2.0)+1, device="cuda", dtype=torch.float32), 
        torch.linspace(-ceil(img.shape[2]*1.41421356237/2.0), ceil(img.shape[2]*1.41421356237/2.0), 100, device="cuda"), 
        torch.linspace(-ceil(img.shape[2]*1.41421356237/2.0), ceil(img.shape[2]*1.41421356237/2.0), 300, device="cuda"), 
        torch.linspace(-ceil(img.shape[2]*1.41421356237/2.0), ceil(img.shape[2]*1.41421356237/2.0), 500, device="cuda"),
        torch.cumsum(torch.softmax(torch.randn((100,), device="cuda"), 0), 0),
        torch.cumsum(torch.softmax(torch.randn((300,), device="cuda"), 0), 0),
        torch.cumsum(torch.softmax(torch.randn((500,), device="cuda"), 0), 0)
    ][i]
    sino = radon.radon_forward(img, positions=positions)
    fsino = radon.radon_filter(sino, radon.ram_lak_filter)
    recon = radon.radon_backward(fsino, img.shape[-1], positions=positions)
    assert torch.abs(recon.min()-img.min()) <= 1e-1, f"Minimum not position quantity invariant: {img.min()} vs {recon.min()}"
    assert torch.abs(recon.max()-img.max()) <= 1e-1, f"Maximum not position quantity invariant: {img.max()} vs {recon.max()}"

In [ ]:
img = next(iter(torchvision.datasets.Caltech256("/data/datasets/", download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Grayscale(), torchvision.transforms.Resize(size, antialias=True), torchvision.transforms.CenterCrop(size)]))))[0].unsqueeze(0).contiguous().to("cuda")
rows, cols = 10, 7
for i in range(6):
    angles = [
        torch.linspace(0.0, torch.pi, 100+1, device="cuda")[:-1], 
        torch.linspace(0.0, torch.pi, 300+1, device="cuda")[:-1], 
        torch.linspace(0.0, torch.pi, 500+1, device="cuda")[:-1],
        (torch.cumsum(torch.softmax(torch.randn((100+1,), device="cuda"), 0), 0)*torch.pi)[:-1],
        (torch.cumsum(torch.softmax(torch.randn((300+1,), device="cuda"), 0), 0)*torch.pi)[:-1],
        (torch.cumsum(torch.softmax(torch.randn((500+1,), device="cuda"), 0), 0)*torch.pi)[:-1]
    ][i]
    sino = radon.radon_forward(img, positions=positions)
    fsino = radon.radon_filter(sino, radon.ram_lak_filter)
    recon = radon.radon_backward(fsino, img.shape[-1], positions=positions)
    assert torch.abs(recon.min()-img.min()) <= 1e-1, f"Minimum not angle quantity invariant: {img.min()} vs {recon.min()}"
    assert torch.abs(recon.max()-img.max()) <= 1e-1, f"Maximum not angle quantity invariant: {img.max()} vs {recon.max()}"